In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# other modules
import numpy as np
from tqdm import trange
import pickle
# internal imports
from utils.attacks.sparse_rs import RSAttack
from utils.models import *
from utils.helpers import *

import torch
import argparse
import json
# PATHING
import os
import sys
root = os.path.abspath(os.curdir)
sys.path.append(root)

from utils.attack import attack
# local imports
# from utils.adv_trainer import adv_trainer

In [2]:
# settings for run
device = 'cuda:0'
net_path = '/home/mbeliaev/home/code/robust-l0/new_trained/test_mnist/cnn_large_k0_p12_seed99/'
# net_path = '/home/mbeliaev/home/code/robust-l0/new_trained/test_cifar/cnn_large_k0_p12_seed99/'
# this contains f_results.p, net.pth, results.p, setup.json
config = json.load(open(net_path+'setup.json','rb'))
print('Loading configuration:')
for key in config.keys():
    print('\t', key,': ', config[key])

# load model
Data = prep_data(root, config['bs'], config['dataset'])
net = Net(config['arch'], config['k'], config['embedding'], Data['x_test'][0].shape)
net = net.to(device)
net.load_state_dict(torch.load(net_path+'net.pth', map_location=device))
net.eval()

Loading configuration:
	 arch :  cnn_large
	 no_adv :  True
	 device :  cuda:3
	 dataset :  MNIST
	 exp :  test_mnist
	 out_dir :  new_trained
	 k :  0
	 perturb :  12
	 seed :  99
	 bs :  128
	 lr :  0.1
	 momentum :  0.9
	 decay :  0.5
	 embedding :  256
	 iters :  5
	 queries :  300
	 epochs :  10


TypeError: 'int' object is not subscriptable

In [ ]:
result_str = pickle.load(open(net_path+'results.p', 'rb'))
for string in result_str: print(string)

In [ ]:
# lets quickly confirm this result
evaluate(net, Data['x_test'], Data['y_test'], device)

In [ ]:
r_acc, x_adv, y_adv, i_adv = attack(net= net,
                            k = 12,
                            x = Data['x_test'],
                            y = Data['y_test'],
                            n_queries=100,
                            n_restarts=1,
                            device=device,
                            log_path=net_path+'rs_log.txt'
                            )

In [ ]:
len(x_adv[0])